## Important:
* You need a cluster with kubernetes>1.8
* You need at least 5 nodes

In [ ]:
SEDLON_API_IP = "35.205.230.74:8080"

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from grpc_python import prediction_pb2
from grpc_python import prediction_pb2_grpc
import grpc

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://{}:8080/oauth/token".format(SELDON_API_IP),
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://{}:8080/api/v0.1/predictions".format(SELDON_API_IP),
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel("{}:5000".format(SELDON_API_IP))
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


In [ ]:
!helm install seldon-core --name seldon-core \
        --set cluster_manager_client_secret=secret \
        --set cluster_manager_service_type=LoadBalancer \
        --set grafana_prom_service_type=LoadBalancer \
        --set apife_service_type=LoadBalancer \
        --set cluster_manager.image.tag=0.3-SNAPSHOT \
        --set apife.image.tag=0.1-SNAPSHOT \
        --set engine.image.tag=0.2-SNAPSHOT

In [ ]:
with open("resources/model.json",'r') as f:
    print f.read()

In [ ]:
!kubectl create -f resources/model_invalid1.json

In [ ]:
!kubectl delete -f resources/model_invalid1.json

In [ ]:
!kubectl create -f resources/model.json

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl apply -f resources/model_with_canary.json

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
!helm delete seldon-core --purge